In [40]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.svm import SVR
import math
from sklearn.model_selection import KFold

In [15]:
# Import CVS
cropsDF = pd.read_csv('../data/agri/frontiers/Processed_Iowa+Cerro+Gordo_1960+2009_Annual+Crop.csv')

# Sort by year
cropsDF = cropsDF.sort_values(by='Year')
cropsDF.head()

# jimg Change Year to YEAR so it matches wx data
cropsDF['YEAR'] = cropsDF.Year
cropsDF.drop('Year', axis=1, inplace=True)

cropsDF.Value = cropsDF.Value.astype('float')

cropsDF

,Program,Period,Geo Level,State,Ag District,County,Commodity,Data Item,Domain Category,Value,YEAR
151,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,19.6,1940
150,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,51.8,1940
148,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,46.5,1941
149,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,14.9,1941
146,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,56.3,1942
...,...,...,...,...,...,...,...,...,...,...,...
4,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,163.1,2013
3,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,50.2,2014
2,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,CORN,"CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,168.3,2014
1,SURVEY,YEAR,COUNTY,IOWA,NORTH CENTRAL,CERRO GORDO,SOYBEANS,"SOYBEANS - YIELD, MEASURED IN BU / ACRE",NOT SPECIFIED,58.9,2015


In [16]:
# Filter data, look for 'BU/ACRE' and filter CORN & SOYBEANS
cornDF = cropsDF.loc[(cropsDF['Data Item'].str.contains('BU / ACRE')) & (cropsDF['Commodity'] == 'CORN')]
beansDF = cropsDF.loc[(cropsDF['Data Item'].str.contains('BU / ACRE')) & (cropsDF['Commodity'] == 'SOYBEANS')]

cornDF = cornDF[['YEAR', 'Value']]
beansDF = beansDF[['YEAR', 'Value']]

print('corn\n', cornDF.head())
print('beans\n', beansDF.head())

corn
      YEAR  Value
150  1940   51.8
148  1941   46.5
146  1942   56.3
144  1943   56.6
142  1944   49.8
beans
      YEAR  Value
151  1940   19.6
149  1941   14.9
147  1942   17.7
145  1943   18.8
143  1944   18.6


In [27]:
# GSP - Growing season precipitation
# GDD - Growing degree days
# GSTmax - Daily max temp avg
# GSTmin - Daily min temp avg
# frost - days < 0 degrees (C)
# summer - days > 25 degrees (C)


weatherDF = pd.read_csv('../data/wx/wx-frontier-agg-3.csv')
weatherDF.head()

# jimg removed averaging, all of these are individual features we need to keep

,Unnamed: 0,YEAR,GSP,GDD,GSTmin,GSTmax,frost,summer,HWI,CWI,dry,wet,PRCP95P
0,1941,1941,3.869512,1555.1,13.099390,25.865244,2,94,21.0,12.0,13,5,4.0
1,1942,1942,3.379878,1327.1,11.957927,24.226220,6,81,11.0,26.0,15,3,5.0
2,1943,1943,3.396341,1303.8,11.908537,23.991463,7,80,12.0,17.0,10,4,2.0
3,1944,1944,3.342331,1467.1,12.899390,24.992073,4,91,17.0,14.0,15,6,2.0
4,1945,1945,3.471951,1130.0,10.643293,23.137195,6,66,10.0,27.0,19,3,5.0


In [28]:
# combine the weather features and the crop yield
combined = weatherDF.merge(cornDF, on='YEAR')
combined['corn'] = combined.Value
combined.drop('Value', axis=1, inplace=True)
combined = combined.merge(beansDF, on='YEAR')
combined['beans'] = combined.Value
combined.drop('Value', axis=1, inplace=True)
combined

,Unnamed: 0,YEAR,GSP,GDD,GSTmin,GSTmax,frost,summer,HWI,CWI,dry,wet,PRCP95P,corn,beans
0,1941,1941,3.869512,1555.100,13.099390,25.865244,2,94,21.0,12.0,13,5,4.0,46.5,14.9
1,1942,1942,3.379878,1327.100,11.957927,24.226220,6,81,11.0,26.0,15,3,5.0,56.3,17.7
2,1943,1943,3.396341,1303.800,11.908537,23.991463,7,80,12.0,17.0,10,4,2.0,56.6,18.8
3,1944,1944,3.342331,1467.100,12.899390,24.992073,4,91,17.0,14.0,15,6,2.0,49.8,18.6
4,1945,1945,3.471951,1130.000,10.643293,23.137195,6,66,10.0,27.0,19,3,5.0,39.8,16.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,2011,2011,2.235671,1428.125,12.427744,24.988415,2,92,20.0,11.0,14,4,0.0,169.3,47.7
71,2012,2012,1.259451,1458.000,11.412805,26.367683,9,107,27.0,15.0,27,3,0.0,124.8,42.5
72,2013,2013,3.569817,1394.850,12.293902,24.716463,4,82,11.0,11.0,21,15,0.0,163.1,40.3
73,2014,2014,3.509146,1235.675,11.458841,23.610366,6,82,2.0,19.0,14,7,0.0,168.3,50.2


In [29]:
# from the paper: "The predictor and response variables were normalized prior 
# to their use by subtracting the mean and dividing by their standard deviation."

feats = ['YEAR', 'GSP', 'GDD', 'GSTmax', 'GSTmin', 'frost', 'summer' ,'HWI', 'CWI', 'dry' ,'wet', 'PRCP95P']
ys = ['corn', 'beans']

def norm(s):
    return (s - s.mean()) / s.std()

for f in feats + ys:
    combined[f] = norm(combined[f])
    
# jimg cross_val_score does not shuffle the data, so make sure we don't get
# test data that's a sequential set of years
combined = combined.sample(frac=1)

combined.head()

,Unnamed: 0,YEAR,GSP,GDD,GSTmin,GSTmax,frost,summer,HWI,CWI,dry,wet,PRCP95P,corn,beans
23,1964,-0.642364,-0.038792,0.040151,-0.383014,0.342699,0.485019,-0.212661,1.319551,0.394591,-0.150868,-0.674822,0.449381,-0.744501,-0.655497
29,1970,-0.367065,0.521201,0.539456,0.810919,0.256343,0.093875,0.422149,0.616993,-0.100629,-0.577854,-0.253058,-0.131715,-0.139778,-0.044621
38,1979,0.045883,0.490805,-0.126349,-0.812513,0.394391,1.267307,0.184095,-0.376410,0.810575,-0.150868,1.012233,0.449381,0.549378,0.291804
41,1982,0.183533,0.495096,-0.519489,0.145216,-0.922841,-0.688414,-1.164877,-0.905218,-0.343286,0.062625,1.433997,-0.131715,0.437561,0.247538
32,1973,-0.229416,-0.087425,1.168906,1.765316,0.549467,-1.861846,0.263446,-0.221545,-0.427473,-1.004840,-0.674822,0.449381,-0.057627,-0.097740


## Corn Data with weather data

In [30]:
# Separating the columns to split the data
X = combined[feats]
y = combined['corn']

# Splitting data into 80% training and 20% testing
# jimg cross_val_score will do this
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [31]:
# Using a SVR model, values
# were taken from the frontiers paper.
# jimg also need linear kernel
model = SVR(kernel='linear', C=0.1, epsilon=0.15)

In [22]:
# jimg cross_val_score will do this
# model.fit(X_train.to_numpy().reshape(-1, 1), y_train.to_numpy())

In [23]:
# Predict values using the testing data
# predictions = model.predict(X_test.to_numpy().reshape(-1, 1))

In [41]:
# Calculate the RMSE
# jimg 'neg_mean_squared_error' is -MSE so negate it
N = 100
srmse = 0
sr2 = 0
for _ in range(N):
    cv = KFold(n_splits=5, shuffle=True)
    rmse = math.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=cv).mean())
    srmse += rmse
    sr2 += r2
    
r2 = sr2 / N
rmse = srmse / N
print('\nr2 %f rmse %f' %(r2,rmse))




r2 0.845479 rmse 0.369619


## SOYBEANS with weather data

In [25]:
# Separating the columns to split the data
X = combined[feats]
y = combined['beans']


In [42]:
# Calculate the RMSE
# jimg 'neg_mean_squared_error' is -MSE so negate it
N = 100
srmse = 0
sr2 = 0
for _ in range(N):
    cv = KFold(n_splits=5, shuffle=True)
    rmse = math.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=cv).mean())
    srmse += rmse
    sr2 += r2
    
r2 = sr2 / N
rmse = srmse / N
print('\nr2 %f rmse %f' %(r2,rmse))


r2 0.845479 rmse 0.369122
